In [ ]:
# pip install yfinance tensorflow scikit-learn pandas joblib
import yfinance as yf, numpy as np, pandas as pd, joblib
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# Télécahrgement des données historiques de Bitcoin

symbol = "BTC-USD"; interval = "1h"; period = "365d"
df = yf.download(symbol, interval=interval, period=period, auto_adjust=True).dropna()
data = df[['Close']].astype('float32').values


[*********************100%***********************]  1 of 1 completed



In [ ]:
# 2) --- Split temporel : train / test (pas de mélange!) ---
train_ratio = 0.85
split = int(len(data)*train_ratio)
train, val = data[:split], data[split:]



In [ ]:
# 3) --- Normalisation (fit sur TRAIN uniquement pour éviter la fuite de données) ---
scaler = MinMaxScaler()
train_s = scaler.fit_transform(train)
val_s   = scaler.transform(val)

look_back = 72
def make_xy(series, L):
    X, y = [], []
    for i in range(L, len(series)):
        X.append(series[i-L:i])
        y.append(series[i])
    return np.array(X), np.array(y)

Xtr, ytr = make_xy(train_s, look_back)
Xva, yva = make_xy(val_s, look_back)

In [ ]:
model = Sequential([
    LSTM(64, input_shape=(look_back,1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(Xtr, ytr, validation_data=(Xva, yva), epochs=15, batch_size=32)

# Sauvegarde
model.save("model.h5")
joblib.dump({"scaler": scaler, "look_back": look_back}, "preproc.pkl")
print("OK: model.h5 + preproc.pkl")
